In [323]:
import json
import time
import pandas as pd
from seleniumwire.utils import decode
from seleniumwire import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [330]:
def load():
    options = {'disable_encoding': True}

    # Create a new instance of the Chrome driver
    driver = webdriver.Chrome(ChromeDriverManager().install(),seleniumwire_options=options )

    # Go to the Google home page
    driver.get("https://food.grab.com/ph/en/")
    return driver

In [332]:
driver = load()



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/benai/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


In [333]:
driver

<seleniumwire.webdriver.Chrome (session="2263575e2f1058affa33fdccc7a50e68")>

In [334]:
def load_more(driver):
    return driver.find_element_by_css_selector("button.ant-btn").click()

In [335]:
input_id = driver.find_element_by_id("location-input").send_keys("pasay")
load_more(driver)

/var/folders/vy/z7v06lns2sqbyp09f_kg__j00000gn/T/ipykernel_72457/1415324294.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  input_id = driver.find_element_by_id("location-input").send_keys("pasay")
/var/folders/vy/z7v06lns2sqbyp09f_kg__j00000gn/T/ipykernel_72457/2217644237.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  return driver.find_element_by_css_selector("button.ant-btn").click()


In [336]:
main_df = pd.DataFrame()

In [337]:
def get_body(request):
    response = request.response
    body_byte = decode(response.body, response.headers.get('Content-Encoding', 'identity'))
    body_dict = json.loads(body_byte.decode("utf-8"))
    return body_dict

In [342]:
def get_popular_restaurant_latlng(popular_restaurants):
    body = get_body(popular_restaurants)
    
    name = []
    lat = []
    lng = []
    
    for i in range(len(body['recommendedMerchantGroups'][0]['recommendedMerchants'])):
        lat.append(body['recommendedMerchantGroups'][0]['recommendedMerchants'][i]['latlng']['latitude'])
        lng.append(body['recommendedMerchantGroups'][0]['recommendedMerchants'][i]['latlng']['longitude'])
        name.append(body['recommendedMerchantGroups'][0]['recommendedMerchants'][i]['address']['name'])
    
    df = pd.DataFrame(list(zip(name, lat, lng)), columns=['Name','latitude','longitude'])
    return df

In [295]:
def get_restaurant_latlng(all_restaurants):
    res_body = get_body(all_restaurants)
    
    name = []
    lat = []
    lng = []
    
    for i in range(len(res_body['searchResult']['searchMerchants'])):
        lat.append(res_body['searchResult']['searchMerchants'][i]['latlng']['latitude'])
        lng.append(res_body['searchResult']['searchMerchants'][i]['latlng']['longitude'])
        name.append(res_body['searchResult']['searchMerchants'][i]['address']['name'])
    
    df = pd.DataFrame(list(zip(name, lat, lng)), columns=['Name','latitude','longitude'])
    return df

In [339]:
all_restaurants = driver.wait_for_request('/foodweb/v2/search')
popular_restaurants = driver.wait_for_request('https://portal.grab.com/foodweb/v2/recommended/merchants')

In [343]:
main_df = main_df.append(get_popular_restaurant_latlng(popular_restaurants))

In [344]:
main_df.reset_index(drop=True)

,Name,latitude,longitude
0,Jollibee - Welcome Rotonda,14.618119,121.001749
1,McDonald's - Sunmall Welcome Rotonda,14.617999,121.001393
2,Starbucks - Pacific Suites Dapitan,14.610519,120.986964
3,Burger King - Welcome Rotonda,14.618641,121.002216
4,Subway - UST Campus [Available for LONG-DISTAN...,14.610538,120.990252
5,Tim Hortons - SM San Lazaro,14.619296,120.985358
6,Tatay Pepe's Tapsilogan - Altura [Available fo...,14.602741,121.010135
7,Chowking - QA Welcome Rotonda,14.618795,121.002419


In [ ]:
has_more = True
while has_more == True:
    df = get_restaurant_latlng(all_restaurants)
    main_df = main_df.append(df)
    
    time.sleep(10)
    try:
        load_more(driver)
    except:
        has_more = False

In [345]:
main_df.to_csv('restaurant_latlng.csv')

In [ ]:
main_df.shape()

In [304]:
driver.quit()